<a href="https://colab.research.google.com/github/VicSkl18/-Engineering-applications/blob/main/Modelador_de_Ciclos_Termodin%C3%A2micos_1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelador de Ciclos Termodinâmicos

###### **Sobre:**

*   Esse programa permite que usuário selecione um processo termodinâmico e através do tipo de gás e dos dados
de pressão e volume iniciais e finais calcular os valores de trabalho(W), calor envolvido (Q) e variação de energia interna (ΔEi) da respectiva transformação. Além disso ele plota o diagrama pxV do processo selecionado ou um ciclo termodinâmico, caso o usário opte por adicionar mais transformações. Nesse caso ele fornecesse os valores de calor e trabalho do ciclo (Qc e Wc) bem como a eficiência (ε) em ciclos que descrevem uma máquina térmica ou coeficiente de desempenho (K) se for um refrigerador.
 Vale lembrar que esse programa é limitado a gerar ciclos de três à quatro processos de transformação termodinâmica.

###### **Dicas para uma boa utilização:**

*   É importante atentar-se ao sistema de unidades utilizado nas entradas do programa, eles irão impactar diretamente nos resultados e unidades das grandezas finais. Fica a cargo do usuário ter pleno conhecimento em análise dimensional.
*   Para garantir uma boa modelagem dos ciclos é importante saber qual tipo de gás será usado, caso contrário, ciclos onde há um ou mais processos adibáticos podem ter sua modelagem comprometida por conta de sua constante. Sempre respeitar a equação: $$\mathrm{p}_\mathrm{i}{\mathrm{V}_\mathrm{i}}^\mathrm{\gamma}\mathrm{\mathrm{=}}\mathrm{p}_\mathrm{f}{\mathrm{V}_\mathrm{f}}^\mathrm{\gamma} $$






###### Bibliotecas:

In [ ]:
from numpy import log as ln
import numpy as np
import plotly.graph_objects as go



###### Função 1: Escolha do tipo de gás

In [ ]:
def gas():
    print("""Selecione um tipo  de gás:

[1] MONOATÔMICO
[2] DIATÔMICO
[3] POLIATÔMICO""")
    print("="*85)
    G = str(input('Qual você deseja? '))
    print("="*85)

    while G not in ["1", "2", "3"]:
        print("Responda apenas com um dos números de cada processo!!!")
        G = str(input('Qual você deseja? '))
        print("="*85)

    if G == "1":
        gama = 1.67
        cv = (3/2)
        cp = (5/2)

    elif G ==  "2":
        gama = 1.4
        cv = (5/2)
        cp = (7/2)


    elif G == "3":
        gama = 1.33
        cv = 3
        cp = 4

    processo(gama, cv, cp)

###### Função 2: Escolha da transformação termodinâmica

In [ ]:
def processo(g, CV, CP):
    print("""Selecione um processo de transformação termodinâmica:

[1] ISOBÁRICO
[2] ISOCÓRICO
[3] ISOTÉRMICO
[4] ADIABÁTICO""")
    print("="*85)
    x = str(input('Qual você deseja? '))
    print("="*85)

    while x not in ["1", "2", "3", "4"]:
        print("Responda apenas com valores referentes a um dos processos!!!")
        x = str(input('Qual você deseja? '))
        print("="*85)



    if x == "1":
        isobarico(g, CV, CP)

    if x == "2":
        isocorico(g, CV, CP)

    if x == "3":
        isotermico(g, CV, CP)

    if x == "4":
        adiabatico(g, CV, CP)

###### Função 3: Processo Isobárico

In [ ]:
def isobarico(gama, cv, cp):

    P, VI, VF = valida_isobarico(gama, cv, cp) #A pressão não varia no processo isobárico, logo, utilizei apenas a inicial
    Pf = P

    #Ticks
    t_x = []
    t_y = []

    esc_x, esc_y = escala(P, Pf, VI, VF, t_x, t_y)

    processo = ['Isobárico']

    #Cálculos: Trabalho, Calor envolvido, Variação de energia interna
    W = P * (VF - VI)
    Q = cp * W
    Ei = cv * P * (VF - VI)

    #Listas para armazenar os dados de W, Q e Ei
    w = []
    q = []
    ei = []
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Ticks
    t_x = []
    t_y = []

    #Resultado dos cálculos
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print("="*85)

    #Diagrama pxV
    pressao = []
    volume = []

    isobarico_x = [VI, VF]
    volume.append(isobarico_x)      #Guardando a lista de dados do processo isobárico numa lista para dados de pressão

    isobarico_y = [P, P]
    pressao.append(isobarico_y)         #Guardando a lista de dados do processo isobárico numa lista para dados de volume

    #Plot do Gráfico

    fig = go.Figure()
    fig.add_trace(go.Scatter(x = isobarico_x, y = isobarico_y, mode = 'lines'))
    fig.update_layout(title='Diagrama pxV: Processo Isobárico',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x,  exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y,  exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

    print("="*85)
    #Verifica se o usuário quer modelar um ciclo ou calcular um novo processo
    x = ciclo_ou_processo()
    if x == 1:
       cont = 1
       processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

###### Função 4: Processo isocórico

In [ ]:
def isocorico(gama, cv, cp):
    PI, PF, V = valida_isocorico(gama, cv, cp) #O volume não varia no processo isocórico, logo, utilizei apenas a inicial
    VI = V

    #Ticks
    t_x = []
    t_y = []

    esc_x, esc_y = escala(PI, PF, VI, V, t_x, t_y)

    processo = ['Isocórico']

    #Cálculos: Trabalho, Calor envolvido, Variação de energia interna
    W = 0
    Q = cv * (PF - PI) * V
    Ei = Q

    #Resultado dos cálculos
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print("="*85)

    #Listas para armazenar os dados de W, Q e Ei
    w = []
    q = []
    ei = []
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Diagrama pxV
    pressao = []
    volume = []

    isocorico_x = [V, V]
    volume.append(isocorico_x)      #Guardando a lista de dados do processo isocórico numa lista para dados de pressão

    isocorico_y = [PI, PF]
    pressao.append(isocorico_y)         #Guardando a lista de dados do processo isocórico numa lista para dados de volume

    #Plot do Gráfico
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = isocorico_x, y = isocorico_y, mode = 'lines'))
    fig.update_layout(title='Diagrama pxV: Processo Isocórico',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

    print("="*85)
    #Verifica se o usuário quer modelar um ciclo ou calcular um novo processo
    x = ciclo_ou_processo()
    if x == 1:
       cont = 1
       processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

###### Função 5: Processo  Isotérmico

In [ ]:
def isotermico(gama, cv, cp):
    PI, PF, VI, VF = valida_isotermico(gama, cv, cp)

    #Ticks
    t_x = []
    t_y = []

    esc_x, esc_y = escala(PI, PF, VI, VF, t_x, t_y)

    processo = ['Isotérmico']

    #Cálculos: Trabalho, Calor envolvido, Variação de energia interna
    W = PF * VF * ln((VF / VI))
    Q = W
    Ei = 0

    #Resultado dos cálculos e nome do processo
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print("="*85)

    #Listas para armazenar os dados de W, Q e Ei
    w = []
    q = []
    ei = []
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Diagrama pxV
    isotermico_pr = []
    isotermico_vl = []
    pressao = []
    volume = []
    k = PI * VI

    if PI > PF:
      isotermico_y = np.arange(PI, PF, (PF-PI) / 20)

    else:
      isotermico_y = np.arange(PF, PI, (PI-PF) / 20)

    for x in isotermico_y:
        isotermico_pr.append(x)

    isotermico_x = k / isotermico_y
    for x in isotermico_x:
        isotermico_vl.append(x)

    if PI > PF:
      isotermico_pr.append(PF)
      isotermico_pr.insert(0, PI)
      isotermico_vl.append(VF)
      isotermico_vl.insert(0, VI)

    else:
      isotermico_pr.append(PI)
      isotermico_pr.insert(0, PF)
      isotermico_vl.append(VI)
      isotermico_vl.insert(0, VF)

    pressao.append(isotermico_pr)
    volume.append(isotermico_vl)

    #Plot do Gráfico
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = isotermico_vl, y = isotermico_pr, mode = 'lines'))
    fig.update_layout(title='Diagrama pxV: Processo Isotérmico',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

    print("="*85)
    #Verifica se o usuário quer modelar um ciclo ou calcular um novo processo
    x = ciclo_ou_processo()
    if x == 1:
       cont = 1
       processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

###### Função 6: Processo adiabático

In [ ]:
def adiabatico(gama, cv, cp):
    PI, PF, VI, VF = valida_adiabatico(gama, cv, cp)

    #Ticks
    t_x = []
    t_y = []

    esc_x, esc_y = escala(PI, PF, VI, VF, t_x, t_y)

    processo = ['Adiabático']

    #Cálculos: Trabalho, calor envolvido, variação de energia interna
    W = (((PI * VI) - (PF * VF)) / (gama - 1))
    Q = 0
    Ei = -W

    #Resultado dos cálculos e nome do processo
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print("="*85)

    #Listas para armazenar os dados de W, Q e Ei
    w = []
    q = []
    ei = []
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Diagrama pxV
    adiabatico_pr = []
    adiabatico_vl = []
    pressao = []
    volume = []
    k = PI * (VI ** gama)

    if PI > PF:
      adiabatico_y = np.arange(PI, PF, (PF - PI) / 20)

    else:
      adiabatico_y = np.arange(PF, PI, (PI - PF) / 20)

    for x in adiabatico_y:
        adiabatico_pr.append(x)

    adiabatico_x = (k /adiabatico_y) ** (1/gama)
    for x in adiabatico_x:
      adiabatico_vl.append(x)

    if PI > PF:
      adiabatico_pr.append(PF)
      adiabatico_pr.insert(0, PI)
      adiabatico_vl.append(VF)
      adiabatico_vl.insert(0, VI)

    else:
      adiabatico_pr.append(PI)
      adiabatico_pr.insert(0, PF)
      adiabatico_vl.append(VI)
      adiabatico_vl.insert(0, VF)

    pressao.append(adiabatico_pr)
    volume.append(adiabatico_vl)

    #Plot do Gráfico
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = adiabatico_vl, y = adiabatico_pr, mode = 'lines'))
    fig.update_layout(title='Diagrama pV: Processo Adiabático',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

    print("="*85)
    #Verifica se o usuário quer modelar um ciclo ou calcular um novo processo
    x = ciclo_ou_processo()
    if x == 1:
       cont = 1
       processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

###### Função 7: Verifica se o usuário quer  modelar um ciclo ou calcular um novo processo

In [ ]:
def ciclo_ou_processo():
    print("""Calcular um novo processo ou modelar um ciclo termodinâmico:

[1] NOVO PROCESSO
[2] CICLO TERMODINÂMICO""")
    print("="*85)
    x = str(input("Qual você deseja? "))
    print("="*85)

    while x not in ["1", "2"]:
        print("""Insira apenas valores referentes às opções!!!""")
        print("""Calcular um novo processo ou modelar um ciclo termodinâmico:
[1]NOVO PROCESSO
[2]CICLO TERMODINÂMICO""")
        x = str(input("Qual você deseja? "))
        print("="*85)

    if x == "1":
        gas()

    elif x == "2":
        return 1

###### Função 8: Teste de validação do processo isobárico

In [ ]:
def valida_isobarico(g, CV, CP):
    global pi, pf, vi, vf

    try:
        pi   = float(input("Insira o valor da pressão inicial: "))
        pf   = float(input("Insira o valor da pressão final: "))
        vi   = float(input("Insira o valor do volume inicial: "))
        vf   = float(input("Insira o valor do volume final: "))
        print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_isobarico(g, CV, CP)

    if pi != pf:
        print("Esse não é um processo isobárico!!!")
        print("Insira novos valores ou retorne ao menu inicial:")
        print("[1]INSERIR NOVOS VALORES\n[2]MENU INICIAL")
        x = str(input("Qual você deseja? "))
        print("="*85)

        while x not in ["1", "2"]:
            print("Insira apenas valores referentes às opções!!!")
            print("Insira novos valores ou retorne ao menu inicial:")
            print("[1]INSERIR NOVOS VALORES\n[2]MENU INICIAL")
            x = str(input("Qual você deseja? "))
            print("="*85)

        if x == "1":
            isobarico(g, CV, CP)

        elif x == "2":
            gas()
    else:
        return pi, vi, vf

###### Função 9: Teste de validação do processo isocórico

In [ ]:
def valida_isocorico(g, CV, CP):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_isocorico(g, CV, CP)

    if vi != vf:
        print("Esse não é um processo isocórico!!!")
        print("Insira novos valores ou retorne ao menu inicial:")
        print("[1]INSERIR NOVOS VALORES\n[2]MENU INICIAL")
        x = str(input("Qual você deseja? "))
        print("="*85)

        while x not in ["1", "2"]:
            print("Insira apenas valores referentes às opções!!!")
            print("Insira novos valores ou retorne ao menu inicial:")
            print("[1]INSERIR NOVOS VALORES\n[2]MENU INICIAL")
            x = str(input("Qual você deseja? "))
            print("="*85)

        if x == "1":
            isocorico(g, CV, CP)

        elif x == "2":
            gas()

    else:
        return pi, pf, vi


###### Função 10: Validação do processo isotérmico

In [ ]:
def valida_isotermico(g, CV, CP):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print('='*60)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_isotermico(g, CV, CP)

    x = valida_isotermico_II(g, CV, CP, pi, pf,  vi, vf)

    if x  == 1:
        return pi, pf, vi, vf

###### Função 11: Validação do processo adiabático

In [ ]:
def valida_adiabatico(g, CV, CP):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_adiabatico(g, CV, CP)

    x = valida_adiabatico_II(g, CV, CP, pi, pf,  vi, vf)

    if x == 1:
        return pi, pf, vi, vf

###### Função 12: Confirma se o processo é isotérmico ou adiabático

In [ ]:
def valida_isotermico_II(gama, cv, cp, PI, PF, VI, VF):
    global pi, pf, vi, vf

    #Abstração da equação: Pi.Vi = Pf.Vf
    PV_i = PI * VI
    PV_f = PF * VF

    if  PV_i > PV_f:
        maior1 = PV_i
        menor1 = PV_f

    elif  PV_i < PV_f:
        maior1 = PV_f
        menor1 = PV_i

    else:
        maior1 = PV_i
        menor1 = PV_f

    #Cálculo do erro percentual entre Pi.Vi e Pf.Vf
    erro_isotermico = (100 *(np.abs(PV_i - PV_f))) / maior1

    #Abstração da equação: (Pi.Vi)^γ = (Pf.Vf)^γ
    PV_i_gama = PI * (VI ** gama)
    PV_f_gama = PF * (VF ** gama)

    if  PV_i_gama > PV_f_gama:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    elif  PV_i_gama < PV_f_gama:
        maior2 = PV_f_gama
        menor2 = PV_i_gama

    else:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    #Cálculo do erro percentual entre (Pi.Vi)^γ e (Pf.Vf)^γ
    erro_adiabático = (100 *(np.abs(PV_i_gama - PV_f_gama))) / maior2

    #Comparação do erro percentual entre as equações:
    if  (PV_i >= PV_f) or (PV_i <= PV_f):
        if erro_isotermico > erro_adiabático:
            print("Esse não é um processo isotérmico!!!")
            print("Insira novos valores ou retorne ao menu inicial:")
            print("[1]INSERIR NOVOS VALORES\n[2]MENU INICIAL")
            x = str(input("Qual você deseja? "))
            print("="*85)

            while x not in ["1", "2"]:
                print("Insira apenas valores referentes às opções!!!")
                print("Insira novos valores ou retorne ao menu inicial:")
                print("[1]INSERIR NOVOS VALORES\n[2]MENU INICIAL")
                x = str(input("Qual você deseja? "))
                print("="*85)

            if x == 1:
                isotermico(gama, cv, cp)

            elif x == 2:
                processo()

        elif (erro_isotermico < erro_adiabático) and (erro_isotermico < 2):
            return 1

        else:
            print("Esse não é um processo isotérmico. Insira novos valores!")
            g = gama
            CV = cv
            CP = cp
            isotermico(g, CV, CP)


            ######




###### Função 13: Confirma se o processo é adiabático ou isotérmico

In [ ]:
def valida_adiabatico_II(gama, cv, cp, PI, PF, VI, VF):
    global pi, pf, vi, vf

    #Abstração da equação: Pi.Vi = Pf.Vf
    PV_i = PI * VI
    PV_f = PF * VF

    if  PV_i > PV_f:
        maior1 = PV_i
        menor1 = PV_f

    elif  PV_i < PV_f:
        maior1 = PV_f
        menor1 = PV_i

    else:
        maior1 = PV_i
        menor1 = PV_f

    #Cálculo do erro percentual entre Pi.Vi e Pf.Vf
    erro_isotermico = (100 *(np.abs(PV_i - PV_f))) / maior1

    #Abstração da equação: (Pi.Vi)^γ = (Pf.Vf)^γ
    PV_i_gama = PI * (VI ** gama)
    PV_f_gama = PF * (VF ** gama)

    if  PV_i_gama > PV_f_gama:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    elif  PV_i_gama < PV_f_gama:
        maior2 = PV_f_gama
        menor2 = PV_i_gama

    else:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    #Cálculo do erro percentual entre (Pi.Vi)^γ e (Pf.Vf)^γ
    erro_adiabático = (100 *(np.abs(PV_i_gama - PV_f_gama))) / maior2

    #Comparação do erro percentual entre as equações:
    if  (PV_i_gama >= PV_f_gama) or (PV_i_gama <= PV_f_gama):
        if erro_isotermico < erro_adiabático:
            print("Esse não é um processo adiabático!!!")
            print("Insira novos valores ou retorne ao menu inicial:")
            print("[1] INSERIR NOVOS VALORES\n[2] MENU INICIAL")
            print("="*85)
            x = str(input("Qual você deseja? "))
            print("="*85)

            while x not in ["1", "2"]:
                print("Insira apenas valores referentes às opções!!!")
                print("Insira novos valores ou retorne ao menu inicial:")
                print("[1] INSERIR NOVOS VALORES\n[2] MENU INICIAL")
                print("="*85)
                x = str(input("Qual você deseja? "))
                print("="*85)

            if x == 1:
                adiabatico(gama, cv, cp)

            elif x == 2:
                processo()

        elif (erro_isotermico > erro_adiabático) and (erro_adiabático < 2):
            return 1

        else:
            print("Esse não é um processo adiabático. Insira novos valores!")
            g = gama
            CV = cv
            CP = cp
            adiabatico(gama, cv, cp)

 A  partir daqui as funções serão apenas adaptações das anteriores mas com alterações que permitam a modelagem de um ciclo termodinâmico.
 Uma das principais diferenças é que foram adicionadas duas variáveis novas que irão "percorrer" entre as funções, uma delas é um contador que vai limitar a quantidade de processos e a outra é uma lista denominada "ciclo_dados", ou como "c_d" em algumas funções. Ademais, não serão plotados gráficos dos processos separados, apenas do ciclo... Os valores de trabalho(W), calor envolvido(Q) e variação de energia interna(ΔEi) também não serão apresentados por enquanto.


###### Função 14 (Ciclo Termodinâmico): Seleção de um novo processo para modelagem de um ciclo

In [ ]:
def processo_ciclo(g, CV, CP, pr, vl, ct, PROCESSO, trabalho, calor, energia, T_x, T_y):
    print("""Selecione um processo de transformação termodinâmica:

[1] ISOBÁRICO
[2] ISOCÓRICO
[3] ISOTÉRMICO
[4] ADIABÁTICO""")
    print("="*85)
    x = str(input('Qual você deseja? '))
    print("="*85)

    while x not in ["1", "2", "3", "4"]:
        print("Responda apenas com um dos números de cada processo!!!")
        x = str(input('Qual você deseja? '))
        print("="*85)

    if x == "1":
        isobarico_ciclo(g, CV, CP, pr, vl, ct, PROCESSO, trabalho, calor, energia, T_x, T_y)

    if x == "2":
        isocorico_ciclo(g, CV, CP, pr, vl, ct, PROCESSO, trabalho, calor, energia, T_x, T_y)

    if x == "3":
        isotermico_ciclo(g, CV, CP, pr, vl, ct, PROCESSO, trabalho, calor, energia, T_x, T_y)

    if x == "4":
        adiabatico_ciclo(g, CV, CP, pr, vl, ct, PROCESSO, trabalho, calor, energia, T_x, T_y)

###### Função 15 (Ciclo Termodinâmico): Processo Isobárico

In [ ]:
def isobarico_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y):

    P, VI, VF = valida_isobarico_ciclo(gama, cv, cp) #A pressão não varia no processo isobárico, logo, utilizei apenas a inicial
    PF = P
    esc_x, esc_y = escala(P, PF, VI, VF, t_x, t_y)

    processo.append('Isobárico')


    #Cálculos: Trabalho, Calor envolvido, Variação de energia interna
    W = P * (VF - VI)
    Q = cp * W
    Ei = cv * P * (VF - VI)

    #Listas para armazenar os dados de W, Q e Ei
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Armazena os dados para plotar o diagrama pxV do ciclo
    VI_I = VI * (1 + 1/2)
    VF_I = VF * (1 + 1/2)
    #isobarico_x = [VI_I, VI, VF, VF_I]
    isobarico_x = [VI, VF]
    volume.append(isobarico_x)      #Guardando a lista de dados do processo isobárico numa lista para dados de pressão

    PI_I = P * (1 + 1/2)
    PF_I = P * (1 + 1/2)
    #isobarico_y = [PI_I, P, P, PF_I]
    isobarico_y = [P, P]
    pressao.append(isobarico_y)         #Guardando a lista de dados do processo isobárico numa lista para dados de volume        #Guardando a lista de dados do processo isobárico numa lista para dados de volume

    cont = cont + 1

    if cont == 2:
      #Plot do Gráfico
      fig = go.Figure()
      fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
      fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
      fig.update_layout(title='Processos Termodinâmicos',
      xaxis_title='Volume',
      yaxis_title='Pressão',
      height = 400, width = 600,
      plot_bgcolor = 'white',
      font = {'family': 'Arial','size': 16,'color': 'black'})
      fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.show()
      print("="*85)

      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 3:
        print("Você já tem 3 processos de transformação termodinâmica,\nvocê deseja plotar um gráfico ou adicionar mais um processo?")
        print('')
        print("[1] PLOTAR UM GRÁFICO\n[2] ADICIONAR MAIS UM PROCESSO")
        print("="*85)
        x = str(input('Qual você deseja? '))
        print("="*85)

        if x == '1':
            #Plot do Gráfico
            fig = go.Figure()
            fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
            fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
            fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
            fig.update_layout(title='Ciclo Termodinâmico',
            xaxis_title='Volume',
            yaxis_title='Pressão',
            height = 400, width = 600,
            plot_bgcolor = 'white',
            font = {'family': 'Arial','size': 16,'color': 'black'})
            fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
            showline=True, linewidth=1, linecolor='black')
            fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
            showline=True, linewidth=1, linecolor='black')
            fig.show()
            print("="*85)

            #Resultado dos cálculos
            cont0 = 0
            for x in range (0, 3, 1):
              cont0 += 1
              print('Processo {}: {}'.format(cont0, processo[x]))
              print("="*85)
              print("W = {:.3f}".format(w[x]))
              print("Q = {:.3f}".format(q[x]))
              print("ΔEi = {:.3f}".format(ei[x]))
              print("="*85)

            eficiencia(w, q)

        elif x == '2':
            #Plot do Gráfico
            fig = go.Figure()
            fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
            fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
            fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
            fig.update_layout(title='Processos Termodinâmico',
            xaxis_title='Volume',
            yaxis_title='Pressão',
            height = 400, width = 600,
            plot_bgcolor = 'white',
            font = {'family': 'Arial','size': 16,'color': 'black'})
            fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
            showline=True, linewidth=1, linecolor='black')
            fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
            showline=True, linewidth=1, linecolor='black')
            fig.show()
            print("="*85)
            processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 4:
          #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[3], y = pressao[3], name = processo[3], mode = 'lines'))
          fig.update_layout(title='Ciclo Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)

          #Resultado dos cálculos
          cont0 = 0
          for x in range (0, 4, 1):
            cont0 += 1
            print('Processo {}: {}'.format(cont0, processo[x]))
            print("="*85)
            print("W = {:.3f}".format(w[x]))
            print("Q = {:.3f}".format(q[x]))
            print("ΔEi = {:.3f}".format(ei[x]))
            print("="*85)

          eficiencia(w, q)
    else:
      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)


###### Função 16 (Ciclo Termodinâmico): Processo Isocórico

In [ ]:
def isocorico_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y):
    PI, PF, V = valida_isocorico_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei) #O volume não varia no processo isocórico, logo, utilizei apenas o inicial
    VF = V
    esc_x, esc_y = escala(PI, PF, V, VF, t_x, t_y)

    processo.append('Isocórico')


    #Cálculos: Trabalho, Calor envolvido, Variação de energia interna
    W = 0
    Q = cv * (PF - PI) * V
    Ei = Q

    #Listas para armazenar os dados de W, Q e Ei
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Armazena os dados para plotar o diagrama pxV do ciclo
    isocorico_x = [V, V]
    volume.append(isocorico_x)      #Guardando a lista de dados do processo isobárico numa lista para dados de pressão

    isocorico_y = [PI, PF]
    pressao.append(isocorico_y)         #Guardando a lista de dados do processo isobárico numa lista para dados de volume

    cont = cont + 1

    if cont == 2:
      #Plot do Gráfico
      fig = go.Figure()
      fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
      fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
      fig.update_layout(title='Processos Termodinâmicos',
      xaxis_title='Volume',
      yaxis_title='Pressão',
      height = 400, width = 600,
      plot_bgcolor = 'white',
      font = {'family': 'Arial','size': 16,'color': 'black'})
      fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.show()
      print("="*85)

      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 3:
          print("Você já tem 3 processos de transformação termodinâmica,\nvocê deseja plotar um gráfico adicionar mais um processo?")
          print('')
          print("[1] PLOTAR UM GRÁFICO\n[2] ADICIONAR MAIS UM PROCESSO")
          print("="*85)
          x = str(input('Qual você deseja? '))
          print("="*85)

          if x == '1':
              #Plot do Gráfico
              fig = go.Figure()
              fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
              fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
              fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
              fig.update_layout(title='Ciclo Termodinâmico',
              xaxis_title='Volume',
              yaxis_title='Pressão',
              height = 400, width = 600,
              plot_bgcolor = 'white',
              font = {'family': 'Arial','size': 16,'color': 'black'})
              fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
              showline=True, linewidth=1, linecolor='black')
              fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
              showline=True, linewidth=1, linecolor='black')
              fig.show()
              print("="*85)

              #Resultado dos cálculos
              cont0 = 0
              for x in range (0, 3, 1):
                cont0 += 1
                print('Processo {}: {}'.format(cont0, processo[x]))
                print("="*85)
                print("W = {:.3f}".format(w[x]))
                print("Q = {:.3f}".format(q[x]))
                print("ΔEi = {:.3f}".format(ei[x]))
                print("="*85)

              eficiencia(w, q)

          elif x == '2':
              #Plot do Gráfico
              fig = go.Figure()
              fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
              fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
              fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
              fig.update_layout(title='Processos Termodinâmico',
              xaxis_title='Volume',
              yaxis_title='Pressão',
              height = 400, width = 600,
              plot_bgcolor = 'white',
              font = {'family': 'Arial','size': 16,'color': 'black'})
              fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
              showline=True, linewidth=1, linecolor='black')
              fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
              showline=True, linewidth=1, linecolor='black')
              fig.show()
              print("="*85)
              processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 4:
          #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[3], y = pressao[3], name = processo[3], mode = 'lines'))
          fig.update_layout(title='Ciclo Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)

          #Resultado dos cálculos
          cont0 = 0
          for x in range (0, 4, 1):
            cont0 += 1
            print('Processo {}: {}'.format(cont0, processo[x]))
            print("="*85)
            print("W = {:.3f}".format(w[x]))
            print("Q = {:.3f}".format(q[x]))
            print("ΔEi = {:.3f}".format(ei[x]))
            print("="*85)

          eficiencia(w, q)

    else:
      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

###### Função 17 (Ciclo Termodinâmico): Processo Isotérmico

In [ ]:
def isotermico_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y):
    PI, PF, VI, VF = valida_isotermico_ciclo(gama, cv, cp)
    esc_x, esc_y = escala(PI, PF, VI, VF, t_x, t_y)

    processo.append('Isotérmico')

    #Cálculos: Trabalho, Calor envolvido, Variação de energia interna
    W = PF * VF * ln((VF / VI))
    Q = W
    Ei = 0

    #Listas para armazenar os dados de W, Q e Ei
    w.append(W)
    q.append(Q)
    ei.append(Ei)

    #Resultado dos cálculos e nome do processo
    print("W = {:.3f}J".format(W))
    print("Q = {:.3f}J".format(Q))
    print("ΔEi = {:.3f}J".format(Ei))
    print("="*85)

    #Armazena os dados para plotar o diagrama pxV do ciclo
    isotermico_pr = []
    isotermico_vl = []
    k = PI * VI

    if PI > PF:
      isotermico_y = np.arange(PI, PF, (PF-PI) / 20)

    else:
      isotermico_y = np.arange(PF, PI, (PI-PF) / 20)

    for x in isotermico_y:
        isotermico_pr.append(x)

    isotermico_x = k / isotermico_y
    for x in isotermico_x:
        isotermico_vl.append(x)

    if PI > PF:
      isotermico_pr.append(PF)
      isotermico_pr.insert(0, PI)
      isotermico_vl.append(VF)
      isotermico_vl.insert(0, VI)

    else:
      isotermico_pr.append(PI)
      isotermico_pr.insert(0, PF)
      isotermico_vl.append(VI)
      isotermico_vl.insert(0, VF)

    pressao.append(isotermico_pr)
    volume.append(isotermico_vl)

    cont = cont + 1

    if cont == 2:
      #Plot do Gráfico
      fig = go.Figure()
      fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
      fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
      fig.update_layout(title='Processos Termodinâmicos',
      xaxis_title='Volume',
      yaxis_title='Pressão',
      height = 400, width = 600,
      plot_bgcolor = 'white',
      font = {'family': 'Arial','size': 16,'color': 'black'})
      fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.show()
      print("="*85)

      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 3:

      print("Você já tem 3 processos de transformação termodinâmica,\nvocê deseja plotar um gráfico adicionar mais um processo?")
      print('')
      print("[1] PLOTAR UM GRÁFICO\n[2] ADICIONAR MAIS UM PROCESSO")
      print("="*85)
      x = str(input('Qual você deseja? '))
      print("="*85)

      if x == '1':
          #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.update_layout(title='Ciclo Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
           showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)

          #Resultado dos cálculos
          cont0 = 0
          for x in range (0, 3, 1):
            cont0 += 1
            print('Processo {}: {}'.format(cont0, processo[x]))
            print("="*85)
            print("W = {:.3f}".format(w[x]))
            print("Q = {:.3f}".format(q[x]))
            print("ΔEi = {:.3f}".format(ei[x]))
            print("="*85)

          eficiencia(w, q)

      elif x == '2':
          #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.update_layout(title='Processos Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)
          processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 4:
      #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[3], y = pressao[3], name = processo[3], mode = 'lines'))
          fig.update_layout(title='Ciclo Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)

          #Resultado dos cálculos
          cont0 = 0
          for x in range (0, 4, 1):
            cont0 += 1
            print('Processo {}: {}'.format(cont0, processo[x]))
            print("="*85)
            print("W = {:.3f}".format(w[x]))
            print("Q = {:.3f}".format(q[x]))
            print("ΔEi = {:.3f}".format(ei[x]))
            print("="*85)

          eficiencia(w, q)

    else:
      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)


###### Função 18 (Ciclo Termodinâmico): Processo adiabático

In [ ]:
def adiabatico_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y):
    PI, PF, VI, VF = valida_adiabatico_ciclo(gama, cv, cp)
    esc_x, esc_y = escala(PI, PF, VI, VF, t_x, t_y)

    processo.append('Adiabático')

    #Cálculos: Trabalho, calor envolvido, variação de energia interna
    W = (((PI * VI) - (PF * VF)) / (gama - 1))
    Q = 0
    Ei = -W

    #Listas para armazenar os dados de W, Q e Ei
    w.append(W)
    q.append(Q)
    ei.append(Ei)


    #Armazena os dados para plotar o diagrama pxV do ciclo
    adiabatico_pr = []
    adiabatico_vl = []

    k = PI * (VI ** gama)

    if PI > PF:
      adiabatico_y = np.arange(PI, PF, (PF - PI) / 20)

    else:
      adiabatico_y = np.arange(PF, PI, (PI - PF) / 20)

    for x in adiabatico_y:
        adiabatico_pr.append(x)

    adiabatico_x = (k /adiabatico_y) ** (1/gama)
    for x in adiabatico_x:
      adiabatico_vl.append(x)

    if PI > PF:
      adiabatico_pr.append(PF)
      adiabatico_pr.insert(0, PI)
      adiabatico_vl.append(VF)
      adiabatico_vl.insert(0, VI)

    else:
      adiabatico_pr.append(PI)
      adiabatico_pr.insert(0, PF)
      adiabatico_vl.append(VI)
      adiabatico_vl.insert(0, VF)

    pressao.append(adiabatico_pr)
    volume.append(adiabatico_vl)

    cont = cont + 1

    if cont == 2:
      #Plot do Gráfico
      fig = go.Figure()
      fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
      fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
      fig.update_layout(title='Processos Termodinâmicos',
      xaxis_title='Volume',
      yaxis_title='Pressão',
      height = 400, width = 600,
      plot_bgcolor = 'white',
      font = {'family': 'Arial','size': 16,'color': 'black'})
      fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
      showline=True, linewidth=1, linecolor='black')
      fig.show()
      print("="*85)

      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 3:
      print("Você já tem 3 processos de transformação termodinâmica,\nvocê deseja plotar um gráfico adicionar mais um processo?")
      print('')
      print("[1] PLOTAR UM GRÁFICO\n[2] ADICIONAR MAIS UM PROCESSO")
      print("="*85)
      x = str(input('Qual você deseja? '))
      print("="*85)

      if x == '1':
          #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.update_layout(title='Ciclo Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)

          #Resultado dos cálculos
          cont0 = 0
          for x in range (0, 3, 1):
            cont0 += 1
            print('Processo {}: {}'.format(cont0, processo[x]))
            print("="*85)
            print("W = {:.3f}".format(w[x]))
            print("Q = {:.3f}".format(q[x]))
            print("ΔEi = {:.3f}".format(ei[x]))
            print("="*85)

          eficiencia(w, q)

      elif x == '2':
          #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.update_layout(title='Processos Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)
          processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

    elif cont == 4:
      #Plot do Gráfico
          fig = go.Figure()
          fig.add_trace(go.Scatter(x = volume[0], y = pressao[0], name = processo[0], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[1], y = pressao[1], name = processo[1], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[2], y = pressao[2], name = processo[2], mode = 'lines'))
          fig.add_trace(go.Scatter(x = volume[3], y = pressao[3], name = processo[3], mode = 'lines'))
          fig.update_layout(title='Ciclo Termodinâmico',
          xaxis_title='Volume',
          yaxis_title='Pressão',
          height = 400, width = 600,
          plot_bgcolor = 'white',
          font = {'family': 'Arial','size': 16,'color': 'black'})
          fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_x, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = esc_y, exponentformat = "power",
          showline=True, linewidth=1, linecolor='black')
          fig.show()
          print("="*85)

          #Resultado dos cálculos
          cont0 = 0
          for x in range (0, 4, 1):
            cont0 += 1
            print('Processo {}: {}'.format(cont0, processo[x]))
            print("="*85)
            print("W = {:.3f}".format(w[x]))
            print("Q = {:.3f}".format(q[x]))
            print("ΔEi = {:.3f}".format(ei[x]))
            print("="*85)

          eficiencia(w, q)

    else:
      processo_ciclo(gama, cv, cp, pressao, volume, cont, processo, w, q, ei, t_x, t_y)

###### Função 19 (Ciclo Termodinâmico): Teste de validação do processo isobárico

In [ ]:
def valida_isobarico_ciclo(g, CV, CP):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_isobarico_ciclo(g, CV, CP)

    if pi != pf:
        print("Esse não é um processo isobárico!!!")
        print("Insira novos valores!!!")
        print("="*85)
        isobarico_ciclo(g, CV, CP)

    else:
        return pi, vi, vf

###### Função 20 (Ciclo Termodinâmico): Teste de validação do processo isocórico

In [ ]:
def valida_isocorico_ciclo(g, CV, CP, PR, VL, Cont, Processo, W, Q, Ei):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_isocorico_ciclo(g, CV, CP, PR, VL, Cont, Processo, W, Q, Ei)

    if vi != vf:
        print("Esse não é um processo isocórico!!!")
        print("Insira novos valores!!!")
        print("="*85)
        isocorico_ciclo(g, CV, CP, PR, VL, Cont, Processo, W, Q, Ei)

    else:
        return pi, pf, vi

###### Função 21 (Ciclo Termodinâmico): Teste de validação do processo isotérmico

In [ ]:
def valida_isotermico_ciclo(g, CV, CP):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_isotermico_ciclo(g, CV, CP, pi, pf, vi, vf)

    x = valida_isotermico_ciclo_II(g, CV, CP, pi, pf,  vi, vf)

    if x  == 1:
        return pi, pf, vi, vf

###### Função 22 (Ciclo Termodinâmico): Teste de validação do processo adiabático

In [ ]:
def valida_adiabatico_ciclo(g, CV, CP):
    global pi, pf, vi, vf

    try:
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: "))
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)

    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        print("="*85)
        valida_adiabatico_ciclo(g, CV, CP, pi, pf, vi, vf)

    x = valida_adiabatico_ciclo_II(g, CV, CP, pi, pf,  vi, vf)

    if x == 1:
        return pi, pf, vi, vf

###### Função 23 (Ciclo Termodinâmico): Teste de validação do processo isotérmico parte II

In [ ]:
def valida_isotermico_ciclo_II(gama, cv, cp, PI, PF, VI, VF):

    #Abstração da equação: Pi.Vi = Pf.Vf
    PV_i = PI * VI
    PV_f = PF * VF

    if  PV_i > PV_f:
        maior1 = PV_i
        menor1 = PV_f

    elif  PV_i < PV_f:
        maior1 = PV_f
        menor1 = PV_i

    else:
        maior1 = PV_i
        menor1 = PV_f

    #Cálculo do erro percentual entre Pi.Vi e Pf.Vf
    erro_isotermico = (100 *(np.abs(PV_i - PV_f))) / maior1

    #Abstração da equação: (Pi.Vi)^γ = (Pf.Vf)^γ
    PV_i_gama = PI * (VI ** gama)
    PV_f_gama = PF * (VF ** gama)

    if  PV_i_gama > PV_f_gama:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    elif  PV_i_gama < PV_f_gama:
        maior2 = PV_f_gama
        menor2 = PV_i_gama

    else:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    #Cálculo do erro percentual entre (Pi.Vi)^γ e (Pf.Vf)^γ
    erro_adiabático = (100 *(np.abs(PV_i_gama - PV_f_gama))) / maior2

    #Comparação do erro percentual entre as equações:
    if  (PV_i >= PV_f) or (PV_i <= PV_f):
        if erro_isotermico > erro_adiabático:
            print("Esse não é um processo isotérmico!!!")
            print("Insira novos valores!!!")
            print("="*85)
            valida_isotermico_ciclo(gama, cv, cp)

        elif (erro_isotermico < erro_adiabático) and (erro_isotermico < 2):
            return 1

        else:
            print("Esse não é um processo isotérmico. Insira novos valores!")
            g = gama
            valida_isotermico_ciclo(g, cv, cp)

###### Função 24 (Ciclo Termodinâmico): Teste de validação do processo adiabático II

In [ ]:
def valida_adiabatico_ciclo_II(gama, cv, cp, PI, PF, VI, VF):

    #Abstração da equação: Pi.Vi = Pf.Vf
    PV_i = PI * VI
    PV_f = PF * VF

    if  PV_i > PV_f:
        maior1 = PV_i
        menor1 = PV_f

    elif  PV_i < PV_f:
        maior1 = PV_f
        menor1 = PV_i

    else:
        maior1 = PV_i
        menor1 = PV_f

    #Cálculo do erro percentual entre Pi.Vi e Pf.Vf
    erro_isotermico = (100 *(np.abs(PV_i - PV_f))) / maior1

    #Abstração da equação: (Pi.Vi)^γ = (Pf.Vf)^γ
    PV_i_gama = PI * (VI ** gama)
    PV_f_gama = PF * (VF ** gama)

    if  PV_i_gama > PV_f_gama:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    elif  PV_i_gama < PV_f_gama:
        maior2 = PV_f_gama
        menor2 = PV_i_gama

    else:
        maior2 = PV_i_gama
        menor2 = PV_f_gama

    #Cálculo do erro percentual entre (Pi.Vi)^γ e (Pf.Vf)^γ
    erro_adiabático = (100 *(np.abs(PV_i_gama - PV_f_gama))) / maior2

    #Comparação do erro percentual entre as equações:
    if  (PV_i_gama >= PV_f_gama) or (PV_i_gama <= PV_f_gama):
        if erro_isotermico < erro_adiabático:
            print("Esse não é um processo adiabático!!!")
            print("Insira novos valores ou retorne ao menu inicial:")
            print("="*85)
            valida_adiabatico_ciclo(g, cv, cp)

        elif (erro_isotermico > erro_adiabático) and (erro_adiabático < 2):
            return 1

        else:
            print("Esse não é um processo adiabático. Insira novos valores!")
            g = gama
            valida_adiabatico_ciclo(g, cv, cp)

###### Função 25: Eficiência e COP


In [ ]:
def eficiencia(W, Q):

   trabalho = sum(W)
   calor = sum(Q)
   print("Wciclo = {:.1f}".format(trabalho))
   print("Qciclo = {:.1f}".format(calor))
   print("="*85)

   Qabs = []
   Qlib = []
   #A teoria diz que Wciclo = Qciclo
   if calor > 0: #Converte calor em trabalho -> Máquina térmica
    for x in range (0, len(Q), 1):
      if Q[x] > 0:
          Qabs.append(Q[x])

      elif Q[x] < 0:
          Qlib.append(Q[x])

    Qf = sum(Qlib)
    Qq = sum(Qabs)
    eficiencia = 1 - (abs(Qf)/abs(Qq))# ε = 1 - (|Qf|/|Qq|)
    print("Máquina térmica: ε = {:.3f}".format(eficiencia))
    print("="*85)

   elif calor < 0:
     for x in range (0, len(Q), 1):
      if Q[x] > 0:
          Qabs.append(Q[x])

      elif Q[x] < 0:
          Qlib.append(Q[x])

     Qf = sum(Qlib)
     Qq = sum(Qabs)
     COP = abs(Qf)/abs(abs(Qq) - abs(Qf))# K = |Qf|/|(|Qq|) - (|Qf|)|
     print("Refrigerador: K = {:.3f}".format(COP))
     print("="*85)

     calcular_novamente()


###### Função 26: Configuração da escala gráfica


In [ ]:
def escala(pi, pf, vi, vf, v_x, p_y):

  y = ((pi + pf) / 2) / 2
  x = ((vi + vf) / 2) / 2

  v_x.append(x)
  p_y.append(y)

  for x in range (1, len(p_y), 1):
    if p_y[x] < (0.2*(p_y[x-1])):
      p_y[x] = p_y[x-1]

    y = p_y[(1-len(p_y))]
    x = v_x[(1-len(p_y))]

  return x, y

###### Função 27: Calcular novamente?


In [ ]:
def calcular_novamente():

   print('''
[1] MODELAR OUTRO CICLO
[2] ENCERRAR PROGRAMA
            ''')
   print("="*85)

   RESP = str(input("Qual opção deseja? "))

   while RESP not in ['1', '2']:

        print("INSIRA APENAS '1' OU '2' !!!")
        RESP = input("Qual opção deseja? ")

   if RESP == '1':
     print("="*85)
     gas()


   elif RESP == '2':
      print("="*85)
      print("CALCULADORA ENCERRADA!!!")
      print("="*85)

######Menu Inicial


In [ ]:
print("="*85)
print("\033[1m CÁLCULO DE PROCESSOS TERMODINÂMICOS \033[0m".center(85))
print("="*85)
print("\033[1;31m• A escolha do sistema de unidades é exclusiva do usuário portanto, cabe ao usuário\nter pleno conhecimento de análise dimensional para uma boa interpretação do gráfico e\ndos resultados.\033[0m")
print("\033[1;31m• Em algumas situações é melhor inserir os valores de pressão e volume em notação\ncientífica!!!\033[0m")
print("Exemplo:\nPara \033[1;32m5.0x10\u2075 Pa\033[0m inserir \033[1;32m5.0e5\033[0m\nPara \033[1;32m3.4x10\u207B\u00B3 m³\033[0m inserir \033[1;32m3.4e-3\033[0m")
print("="*85)

gas() #Chama a função de seleção do tipo de gás